In [27]:
## colab 환경 설정
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
## colab 환경 설정
import os
from pathlib import Path

folder = "colab/kobert_practice" ## 자기 드라이브 경로 입력

base_path = Path("/content/gdrive/My Drive/")
project_path = base_path / folder
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():ㅐ무
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"{os.getcwd()}")

/content/gdrive/My Drive/colab/kobert_practice


# 필요 패키지 설치
colab 기준으로 패키지 설치
local이나 서버 환경에서는 바뀔 수 있으니 주의 필요

In [3]:
!git clone https://github.com/SKTBrain/KoBERT.git

fatal: destination path 'KoBERT' already exists and is not an empty directory.


In [4]:
cd KoBERT

/content/gdrive/My Drive/colab/kobert_practice/KoBERT


In [6]:
!pip install -r requirements.txt
!pip install .
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |████████████████████████████████| 68.7MB 43kB/s 
     |████████████████████████████████| 256kB 37.4MB/s 
     |████████████████████████████████| 1.1MB 33.9MB/s 
     |████████████████████████████████| 3.9MB 36.8MB/s 
     |████████████████████████████████| 675kB 40.5MB/s 
     |████████████████████████████████| 7.4MB 15.9MB/s 
     |████████████████████████████████| 890kB 36.2MB/s 
     |████████████████████████████████| 3.8MB 35.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470044 sha256=18c2024d0c2a27e99fdbe7ad895049f62eb61d8f1cb7d6affcd7dbb9e359042f
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3b8fc7c785c8ff57f24ae0f6acbb0ab3e12dccc4f5fef1fab78e1a3f40abf09c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully

In [7]:
cd ..

/content/gdrive/My Drive/colab/kobert_practice


# 모델

In [9]:
## 필요 패키지 로드
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.pytorch_kobert import get_pytorch_kobert_model
import pandas as pd
import numpy as np
from kobert.utils import get_tokenizer
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [23]:
## bert 모델 불러오기, vocab 불러오기, tokenizer 불러오기..
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

## 필요한 파라미터
vocab_size = len(vocab.idx_to_token)
max_len = 64

using cached model
using cached model
using cached model


In [14]:
## 모델 클래스
class BERTseq2seq(nn.Module):
  def __init__(self, bert, hidden_size = 768, vocab_size = 8002, dr_rate=None, params=None):
    super(BERTseq2seq, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

    self.linear = nn.Linear(hidden_size, vocab_size)
    
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
        attention_mask[i][:v] = 1
    return attention_mask.float()
  
  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    seq, _ = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    # if self.dr_rate:
    #   out = self.dropout(seq)
    seq = self.linear(seq)

    return seq

In [15]:
## 모델 초기화
model = BERTseq2seq(bertmodel, vocab_size=vocab_size, dr_rate=0.5).to(device)

In [20]:
## 모델 불러오기
save_path = '/content/gdrive/My Drive/colab/kobert_practice/model/sample_model.tar'
checkpoint = torch.load(save_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [21]:
## 입력된 텍스트에 대한 답변 리턴
def translate(s):
  transform = nlp.data.BERTSentenceTransform(tok, max_seq_length=max_len, pad=True, pair=False)
  token, length, segment = transform([s])
  token = torch.from_numpy(token).to(device)
  length = torch.from_numpy(length).to(device)
  segment = torch.from_numpy(segment).to(device)
  model.eval()
  token = token.view(1,-1)
  length = length.view(1, -1)
  segment = segment.view(1, -1)
  seq = model(token.long(), length, segment.long())
  _, topi = seq.topk(1)
  answer = ""
  for x in topi[0]:
    answer += vocab.idx_to_token[x.item()]
    if x.item() in [1,3]:
      break
  return answer

In [26]:
translate("여기에 입력")

'[CLS]▁많은▁안[PAD]'